## Capstone Project
-------

### Stage 1 - Cleaning phase
------

#### Importing packages and data
------

In [1]:
# import packages
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn import pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [2]:
# import data
data_2017 = pd.read_csv("Datasets/2017_survey.csv")
data_2018 = pd.read_csv("Datasets/2018_survey.csv")

data = pd.concat([data_2017,data_2018],sort=False,ignore_index=True)

#### Preliminary data cleaning
------

<u> Goals: </u>

1. Cleaning column titles
2. Combining and cleaning responses
3. Handling NaN values

In [3]:
def combine_columns(first_num,num_list,df_name):
    '''
    This function combines duplicate columns.
    
    Inputs:
    ------
    first_num: an integer of the column number you wish the information to be combined to
    num_list: a list of integers of the column numbers you wish the information to be combined
    df_name = the name of the dataframe
    
    '''
    for num in num_list:
        df_name.iloc[:,first_num] = df_name.iloc[:,first_num] + df_name.iloc[:,num]

def combine_info(my_list,column_name = "What is your race?"):
    '''
    This function combines similar responses (but spelled differently or used slightly different wording) 
    into one category of response.
    
    Inputs:
    ------
    my_list = a list of responses you want to put into the category
    column_name = the name of the column
    
    '''
    for num,info in enumerate(my_list):
        if num > 0:
            df[column_name][df[column_name]==info] = my_list[0]
            
def fillna_with_median(question = "What is your age?"):
    '''
    This function fills NaN values with the median of the column.
    
    Input:
    ------
    question: the name of the column
    
    '''
    median = np.median(df[question][df[question].isna()==False])
    df[question].fillna(median,inplace=True)

In [4]:
# cleaning column titles
columns_to_clean = data.columns[data.columns.str.contains("<strong>")]

list_1 = (columns_to_clean[:7],columns_to_clean[9:11])
list_2 = (columns_to_clean[-5],columns_to_clean[-3])
list_3 = (columns_to_clean[-4],columns_to_clean[-2])

for item in list_1:
    for question in range(len(item)):
        data.rename(columns = {f"{item[question]}": f"{item[question][8:-9]}"},inplace=True)

for question in list_2:
    data.rename(columns = {f"{question}": f"{question[:20]+question[28:32]+question[-4:]}"},inplace=True)

for question in list_3:
    data.rename(columns = {f"{question}": f"{question[:34]+question[42:46]+question[-4:]}"},inplace=True)

my_list = [["If you have a mental health disorder, how often do you feel that it interferes with your work <strong>when being treated effectively?</strong>",
            "If you have a mental health disorder, how often do you feel that it interferes with your work when being treated effectively?"],
           ["If you have a mental health disorder, how often do you feel that it interferes with your work <strong>when <em>NOT</em> being treated effectively (i.e., when you are experiencing symptoms)?</strong>",
            "If you have a mental health disorder, how often do you feel that it interferes with your work when NOT being treated effectively (i.e., when you are experiencing symptoms)?"]]

for pairs in my_list:
    data.rename(columns = {pairs[0]:pairs[1]},inplace=True)

In [5]:
# insert unique id and drop column "#"
data.insert(0,"id",(data.index+1))

data.drop(columns = "#",inplace=True)

##### Cleaning responses for MH disorders
-----

In [6]:
# combining and cleaning disorders
start_num = 50

while start_num < 64:
    data.iloc[:,start_num].fillna(data.iloc[:,(start_num+13)],inplace=True)
    data.iloc[:,start_num].fillna(data.iloc[:,(start_num+26)],inplace=True)
    data.iloc[:,start_num].fillna(0,inplace=True)
    data.iloc[:,start_num].where(data.iloc[:,start_num]==0,1,inplace=True)
    start_num += 1

In [7]:
# checking number of entries in each column
others_dummy = pd.concat([pd.get_dummies(data["Other.1"]),pd.get_dummies(data["Other.2"])],axis=1)

In [8]:
# combining columns
ADHD_list = (5,8)
ASD_list = (1,2,3,7,9)
Depression_list = (15,16)

combine_columns(0,ADHD_list,others_dummy)
combine_columns(10,ASD_list,others_dummy)
combine_columns(14,Depression_list,others_dummy)

others_dummy.iloc[:,19] = others_dummy.iloc[:,19] + others_dummy.iloc[:,20]

In [9]:
# dropping duplicate columns in others_dummy
drop_list = list(ADHD_list + ASD_list + Depression_list)
drop_list.append(20)
column_names = []

for num in drop_list:
    column_names.append(others_dummy.columns[num])

others_dummy.drop(columns = column_names,inplace=True)

In [10]:
# dropping duplicate columns in data
data.drop(columns=data.columns[62:89],inplace=True)

In [11]:
# Mood disorder
data.iloc[:,51] = data.iloc[:,51] + others_dummy.iloc[:,-5] + others_dummy.iloc[:,4]

# ADHD
data.iloc[:,54] += others_dummy.iloc[:,0]

# How would team members react to your MH dx
data.iloc[:,76].fillna(data.iloc[:,-2],inplace=True)
data.drop(columns=data.columns[-2],inplace=True)

# Current employer provide MH coverage
data.iloc[:,5].fillna(data.iloc[:,-3],inplace=True)
data.drop(columns=data.columns[-3],inplace=True)

# drop last column in data
data.drop(columns=data.columns[-1],inplace=True)

# drop columns in others_dummy
drop_list2 = (0,4,7)
column_names2 = []

for num in drop_list2:
    column_names2.append(others_dummy.columns[num])

others_dummy.drop(columns = column_names2,inplace=True)

In [12]:
# interfering with work when MH not tx properly
data.iloc[:,66].fillna(data.iloc[:,-1],inplace=True)
data.drop(columns=data.columns[-1],inplace=True)

##### Handling NaN values
------

In [13]:
# combining df
data = pd.concat([data.iloc[:,:62], others_dummy, data.iloc[:,62:]],axis = 1)

In [14]:
# making a copy of the dataset
df = data.copy()

In [15]:
# fill in some NaN for some columns that have over 50% NaN value to keep those columns
for num in [49,-6]:
    df.iloc[:,num].fillna("Did not answer",inplace=True)

In [16]:
# drop columns with over 50% NaN values
delete_list = df.isna().sum()[df.isna().sum() > 587]

for num in range(len(delete_list)):
    df.drop(columns = delete_list.index[num],inplace=True)

In [17]:
# drop columns with over 25% NaN values that are deemed not essential
df.drop(columns=df.columns[-12],inplace=True)

In [18]:
# fillna for describing things to improve
df.iloc[:,-12].fillna("Did not answer",inplace=True)

In [19]:
# cleaning race column
question = "What is your race?"
df[question].fillna(df["Other.3"],inplace=True)

In [20]:
# cleaning up the responses
hispanics = ["Hispanic","Hispanic or Latino","Latina","Latino","Latinx","mexican american "]
no_answer = ["Did not answer","I prefer not to answer","I am of the race of Adam, the first human."]
mixed = ["Mixed","More than one of the above","Hispanic, White","Mestizo"]
jewish = ["Jewish","Ashkenazi"]
caucasian = ["Caucasian","White","European American","My race is white, but my ethnicity is Latin American"]
caribbean = ["Caribbean","Indo-Caribbean","West Indian"]
asian = ["Asian","South Asian"]
aa = ["Afrcian American","Black or African American"]

race_list = [hispanics,no_answer,mixed,jewish,caucasian,caribbean,asian,aa]

for race in race_list:
    combine_info(race,column_name = "What is your race?")

In [21]:
# dropping duplicate column
df.drop(columns="Other.3",inplace=True)

In [22]:
# Cleaning gender
question = "What is your gender?"

df[question].fillna("Did not answer",inplace=True)

In [23]:
# Combine gender responses
male = ["Male","Cis Male","Cis male","Cis-male","Cisgender male","M","MALE","cis hetero male","cis male",
        "cis male ","cis-male","dude","m","male","male (hey this is the tech industry you're talking about)",
        "male, born with xy chromosoms","male/androgynous","man","God King of the Valajar","Mail","Male ",
        "Male (cis)","Male, cis","SWM","Malel","Man","Ostensibly Male"]

female = ["Female","*shrug emoji* (F)","Cis female ","Cis woman","Cis-Female","Cisgendered woman","F",
          "F, cisgender","Female ","Female (cis) ","Female (cisgender)","I identify as female","Woman",
          "Woman-identified","cis female","cis-Female","cisgender female","f","femail","female",
          "female (cis)","female (cisgender)","femalw","woman","My sex is female."]

genderqueer = ["Genderqueer","Agender","Agender/genderfluid","Contextual","Female-ish","Demiguy",
               "Female/gender non-binary.","Genderfluid","Genderqueer demigirl","Genderqueer/non-binary",
               "Male (or female, or both)","Male-ish","NB","Non binary","Non-binary","Nonbinary",
               "Nonbinary/femme","She/her/they/them","gender non-conforming woman","genderfluid",
               "non binary","non-binary","nonbinary","uhhhhhhhhh fem genderqueer?","male/androgynous "]

transgender = ["Transgender","Trans female","Trans man","Trans woman","Transfeminine",
               "trans woman","transgender"]

other = ["Other","None","\-","none","sometimes"]

gender_list = [male,female,genderqueer,transgender,other]

for gender in gender_list:
    combine_info(gender,column_name = "What is your gender?")

In [24]:
# Clean up # of employees
question = "How many employees does your company or organization have?"

df[question].fillna(0,inplace=True)

In [25]:
# grabbing a list of participants who are self-employed
self_employed = df[df[question]==0].index.values

In [26]:
# check and see if # NaN = 169 are all from self-employed participants
my_list = df.isna().sum().index[df.isna().sum()==169]
index_list = list(my_list.values)

for i,j in enumerate(index_list):
    b = df[index_list[i]][df[index_list[i]].isna()==True].index.values
    print(np.bincount(self_employed == b))

# looks like all these are from the self-employed group

[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]
[  0 169]


In [27]:
# changing NaN values in columns with text data to NA
my_list = df.isna().sum().index[5:13]
column_list = list(my_list.values)
column_list.append(df.isna().sum().index[14])

for question in column_list:
    df.loc[(self_employed),column_list]="Not Applicable"

In [28]:
# Fill NaNs in age and overall rating with median
fillna_with_median(question = "What is your age?")
fillna_with_median(question = "Overall, how well do you think the tech industry supports employees with mental health issues?")

In [29]:
# Using Network ID as a clue to fill in a NaN value
df[df["What country do you live in?"].isna()==True]

,id,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Have you ever discussed your mental health with your employer?,Would you feel comfortable discussing a mental health issue with your coworkers?,Have you ever discussed your mental health with coworkers?,Have you ever had a coworker discuss their or another coworker's mental health with you?,"Overall, how much importance does your employer place on physical health?","Overall, how much importance does your employer place on mental health?",Do you have previous employers?,Was your employer primarily a tech company/organization?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health disorders and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Did you ever discuss your mental health with your previous employer?,Would you have been willing to discuss your mental health with your coworkers at previous employers?,Did you ever discuss your mental health with a previous coworker(s)?,Did you ever have a previous coworker discuss their or another coworker's mental health with you?,"Overall, how much importance did your previous employer place on physical health?","Overall, how much importance did your previous employer place on mental health?",Do you currently have a mental health disorder?,Have you ever been diagnosed with a mental health disorder?,"Anxiety Disorder (Generalized, Social, Phobia, etc)","Mood Disorder (Depression, Bipolar Disorder, etc)","Psychotic Disorder (Schizophrenia, Schizoaffective, etc)","Eating Disorder (Anorexia, Bulimia, etc)",Attention Deficit Hyperactivity Disorder,"Personality Disorder (Borderline, Antisocial, Paranoid, etc)",Obsessive-Compulsive Disorder,Post-Traumatic Stress Disorder,Stress Response Syndromes,Dissociative Disorder,Substance Use Disorder,Addictive Disorder,Suicidal,Adjustment disorder,Autism Spectrum Disorder,Codependence,Cyclothymia,Gender Dysphoria,Multiple Sclerosis & Mental Health,Panic Disorder,Tourette's,Have you had a mental health disorder in the past?,Have you ever sought treatment for a mental health disorder from a mental health professional?,Do you have a family history of mental illness?,"If you have a mental health disorder, how often do you feel that it interferes with your work when being treated effectively?","If you have a mental health disorder, how often do you fe

In [30]:
# Using Network ID as a clue to fill in a NaN value
df.loc[755,"What country do you live in?"]="Did not answer"

for num in [-8,-5]:
    df.iloc[753,num]="United States of America"
    
for num in [-7,-4]:
    df.iloc[753,num]="Indiana"

In [31]:
# Fill in NaN values for US states
df["What US state or territory do you live in?"].fillna("NA",inplace=True)

In [32]:
# Dropping columns
df.drop(columns = df.columns[-5:-3],inplace=True)

In [33]:
# Filling some NaN with -1 - indicating did not answer
my_list = df.isna().sum()[df.isna().sum() > 1].index
positions = [0,1,3,4,5,8,-3,-4,-7,-8,-11,-12,-14]

for i in positions:
    df.loc[:,my_list[i]].fillna(-1,inplace=True)

##### Filling in the missing values in overall ratings using machine learning algorithm
------

<u>Independent variables:</u> 
- Gender
- Country of residence
- Race

<u>Dependent variable:</u>
- Respective overall ratings

<u>Models considered:</u>
- Decision Tree Classifier
- Random Forrest Classifier
- XGB Classifier

In [34]:
# creating a list of questions with overall ratings
rating_list = df.isna().sum().index[df.isna().sum().index.str.contains("Overall")].values

In [35]:
# creating independent and dependent variables for the model
original = {}
final = {}
train = {}
test = {}

for num in range(len(rating_list)-1):
    original[num] = df.loc[:,(rating_list[num],"What is your gender?","What country do you live in?",
                              "What is your race?")]
    
    dummies = pd.get_dummies(original[num].iloc[:,1:])
    final[num] = pd.concat([original[num].iloc[:,0],dummies],axis=1)

    train[num] = final[num][final[num].iloc[:,0].isna()==False]
    test[num] = final[num][final[num].iloc[:,0].isna()==True]

#overall industry ratings need to be dealt with separately

In [36]:
#Using GridsearchCV to determine the optimum model for each rating

for num in range(len(rating_list)-1):
    x = train[num].iloc[:,1:]
    y = train[num].iloc[:,0]
    x_test = test[num].iloc[:,1:]

    estimators = [('model', DecisionTreeClassifier())]

    pipe = pipeline.Pipeline(estimators)

    param_grid = [{'model': [DecisionTreeClassifier()]},
                  {'model': [RandomForestClassifier()]},
                  {'model': [XGBClassifier()]}]

    grid = GridSearchCV(pipe, param_grid, cv=5)
    grid_search = grid.fit(x, y)
    print(num,grid_search.best_estimator_)

/Users/nattiechan/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0 Pipeline(memory=None,
     steps=[('model', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
1 Pipeline(memory=None,
     steps=[('model', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])
2 Pipeline(memory=None,
     steps=[('model', XGBClassifier(base_score=0.5, booster='gbtree

/Users/nattiechan/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


3 Pipeline(memory=None,
     steps=[('model', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])


Based on results from GridSearchCV, Decision Tree Classifier is best for the first question on the list and XGB Classifier is the best for the rest of the questions.

In [37]:
# Filling in the NaN values
for num in range(len(rating_list)-1):
    x = train[num].iloc[:,1:]
    y = train[num].iloc[:,0]
    x_test = test[num].iloc[:,1:]

    if num == 0:
        dt = DecisionTreeClassifier()
        # 5-fold cross-validated to be the best one out of the box

        dt.fit(x,y)
        results = dt.predict(x_test)
    
    else:
        xgb = XGBClassifier()
        # 5-fold cross-validated to be the best one out of the box

        xgb.fit(x,y)
        results = xgb.predict(x_test)
    
    values = df.loc[:,rating_list[num]][df[rating_list[num]].isna()==True].index.values

    for position, value in enumerate(values):
        df.loc[value,rating_list[num]] = results[position]

In [38]:
# Fill in more NaN values
column_list = df.isna().sum()[df.isna().sum() > 1].index

for column in column_list:
    df.loc[:,column].fillna("Did not answer",inplace=True)

##### Simplify responses for MH disorders
------

In [39]:
# Combining responses
neuro = ["Attention Deficit Hyperactivity Disorder","Autism Spectrum Disorder","Tourette's"]
adjust = ["Adjustment disorder","Stress Response Syndromes"]
substance = ["Substance Use Disorder","Addictive Disorder"]
anxiety = ["Anxiety Disorder (Generalized, Social, Phobia, etc)","Panic Disorder"]
mood = ["Mood Disorder (Depression, Bipolar Disorder, etc)","Cyclothymia"]
other = ['Suicidal','Codependence','Gender Dysphoria', 'Multiple Sclerosis & Mental Health']

column_list = [neuro,adjust,substance,anxiety,mood,other]

for var in column_list:
    for num,column in enumerate(var):
        if num > 0:
            df.loc[:,var[0]] += df.loc[:,var[num]]
            df.drop(columns = var[num],inplace=True)

In [40]:
# renaming some columns

my_list = [["Attention Deficit Hyperactivity Disorder","Neurodevelopmental Disorders"],
           ["Substance Use Disorder","Substance-Related and Addictive Disorders"],
           ["Suicidal","Other"],
           ["Anxiety Disorder (Generalized, Social, Phobia, etc)","Anxiety Disorder"],
           ["Mood Disorder (Depression, Bipolar Disorder, etc)","Mood Disorder"],
           ["Psychotic Disorder (Schizophrenia, Schizoaffective, etc)","Psychotic Disorder"],
           ["Eating Disorder (Anorexia, Bulimia, etc)","Eating Disorder"],
           ["Personality Disorder (Borderline, Antisocial, Paranoid, etc)","Personality Disorder"]]

for pairs in my_list:
    df.rename(columns = {pairs[0] : pairs[1]},inplace=True)

In [41]:
# replacing some duplicate responds since the answers are binary (0/1)
for num in range(36,48):
    df.iloc[:,num].replace(2,1,inplace=True)

### Advanced cleaning
------

Model-specific data cleaning and preparation.

In [42]:
question = "Would you have been willing to discuss your mental health with your coworkers at previous employers?"
old_answer = "At some of my previous employers"
new_answer = "Some of my previous employers"

df.loc[:,question][df.loc[:,question]==old_answer]=new_answer

new_name = "Does your employer provide mental health benefits as part of healthcare coverage?"

df.rename(columns = {df.columns[5] : new_name}, inplace=True)

In [43]:
question_1 = "Does your employer provide mental health benefits as part of healthcare coverage?"
question_2 = "Do you know the options for mental health care available under your employer-provided health coverage?"
question_3 = "Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?"
question_4 = "Have you observed or experienced supportive or well handled response to a mental health issue in your current or previous workplace?"

answer_1 = "Not Applicable"
answer_2 = "Not eligible for coverage / NA"
answer_3 = "Did not answer"
answer_4 = "Yes, I experienced"
answer_5 = "Yes, I observed"
answer_6 = "Yes"

df.loc[:,question_1][df.loc[:,question_1]==answer_1]=answer_2

df.loc[:,question_2][df.loc[:,question_2]==answer_3]=answer_1

for question in [question_3,question_4]:
    df.loc[:,question][df.loc[:,question]==answer_4]=answer_5
    df.loc[:,question][df.loc[:,question]==answer_5]=answer_6

In [44]:
df.head()

,id,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Have you ever discussed your mental health with your employer?,Would you feel comfortable discussing a mental health issue with your coworkers?,Have you ever discussed your mental health with coworkers?,Have you ever had a coworker discuss their or another coworker's mental health with you?,"Overall, how much importance does your employer place on physical health?","Overall, how much importance does your employer place on mental health?",Do you have previous employers?,Was your employer primarily a tech company/organization?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health disorders and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Did you ever discuss your mental health with your previous employer?,Would you have been willing to discuss your mental health with your coworkers at previous employers?,Did you ever discuss your mental health with a previous coworker(s)?,Did you ever have a previous coworker discuss their or another coworker's mental health with you?,"Overall, how much importance did your previous employer place on physical health?","Overall, how much importance did your previous employer place on mental health?",Do you currently have a mental health disorder?,Have you ever been diagnosed with a mental health disorder?,Anxiety Disorder,Mood Disorder,Psychotic Disorder,Eating Disorder,Neurodevelopmental Disorders,Personality Disorder,Obsessive-Compulsive Disorder,Post-Traumatic Stress Disorder,Dissociative Disorder,Substance-Related and Addictive Disorders,Other,Adjustment disorder,Have you had a mental health disorder in the past?,Have you ever sought treatment for a mental health disorder from a mental health professional?,Do you have a family history of mental illness?,"If you have a mental health disorder, how often do you feel that it interferes with your work when being treated effectively?","If you have a mental health disorder, how often do you feel that it interferes with your work when NOT being treated effectively (i.e., when you are experiencing symptoms)?",Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?,How willing would you be to share with friends and family that

------
### Please continue to Capstone modelling stage v.3 for the remainder of the project.
------